<font size="+3"><strong>Exploring the Data</strong></font>

In this project, we're going to work with data from the [Funda](https://funda.nl). This company provides houses pricing in the Netherlands.

Keep in mind that **Funda** prices are just a starting point and properties usually sold for 110% of their price or even more than 120% due to a auction nature.
During this auction, you should not know bids of others, but this is not true, because agents sometimes share current bids with their clients (this is not legal, but happens).
In addition, lower bid can win, because the person is not going to take out any loans (cash is ready to be transferred).

Additional expenses usually around 7%, but this is just additional information about property market in the Netherlands.

In [5]:
import pandas as pd

# Prepare data

First, we need to load data. At this pount you already should have dump of data from [Funda](https://funda.nl) (please follow the README file of this project).

In [8]:
df = pd.read_json('../scrapy/x.json_backup')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27558 entries, 0 to 27557
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   url                   27558 non-null  object
 1   property_type         27558 non-null  object
 2   town                  27558 non-null  object
 3   address               27558 non-null  object
 4   postcode              27558 non-null  object
 5   price                 27558 non-null  object
 6   building_type         27558 non-null  object
 7   living_area           27558 non-null  object
 8   year_of_construction  26739 non-null  object
dtypes: object(9)
memory usage: 1.9+ MB
